# Reweighting

This page examines the effects of reweighting the TMD data file along several key variables.

## Demogaphics

In [1]:
import pandas as pd
from tax_microdata_benchmarking.storage import STORAGE_FOLDER

df = pd.read_csv(STORAGE_FOLDER / "output" / "tmd_2021.csv")

COLUMNS = [
    "nu06",
    "nu13",
    "nu18",
]

pre_reweighting_sums = [
    (df[column] * df.s006_original).sum() / 1e6 for column in COLUMNS
]
post_reweighting_sums = [
    (df[column] * df.s006).sum() / 1e6 for column in COLUMNS
]

pd.DataFrame(
    {
        "Variable": COLUMNS,
        "Pre-reweighting": pre_reweighting_sums,
        "Post-reweighting": post_reweighting_sums,
    }
).round(1)

,Variable,Pre-reweighting,Post-reweighting
0,nu06,35.1,30.7
1,nu13,66.6,57.4
2,nu18,87.2,74.8


In [7]:
from microdf import MicroDataFrame

df = MicroDataFrame(df, weights="s006_original")

df.nu18.groupby(df.data_source).sum() / 1e6

data_source
0.0     6.051887
1.0    81.169796
dtype: float64

In [9]:
from policyengine_us import Microsimulation

sim = Microsimulation(dataset="cps_2021")

Downloaded ASEC: 100%|██████████| 158M/158M [00:43<00:00, 3.64MiB/s]  


No previous year data available for this dataset, skipping previous year income imputation.


In [10]:
(sim.calculate("age", period=2021) < 18).sum() / 1e6

73.12326943637085

In [8]:
from microdf import MicroDataFrame

df = MicroDataFrame(df, weights="s006")

df.nu18.groupby(df.data_source).sum() / 1e6

data_source
0.0     6.051887
1.0    68.759702
dtype: float64

# QBI deduction

In [2]:
pd.DataFrame(
    {
        "Pre-reweighting": [(df.s006_original * df.qbided).sum() / 1e9],
        "Post-reweighting": [(df.s006 * df.qbided).sum() / 1e9],
    }
).round(1)

,Pre-reweighting,Post-reweighting
0,242.3,203.0


## SALT distributions

In [3]:
pd.DataFrame(
    {
        "Pre-reweighting": [
            (df.s006_original * (df.e18400 + df.e18500)).sum() / 1e9
        ],
        "Post-reweighting": [(df.s006 * (df.e18400 + df.e18500)).sum() / 1e9],
    }
).round(1)

,Pre-reweighting,Post-reweighting
0,346.4,317.1
